In [1]:
import pandas as pd
from pandas import DataFrame
import numpy as np
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot as plt
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import os
path='E:\【ML_DL】\【服装项目】\dataset\shopN\data\shop_'

Using TensorFlow backend.


In [2]:
shop_stop_20181029=[]
s2=[]
for i in range(1,639):
    df = pd.read_csv(path+str(i)+'.csv',low_memory=False)
    if df['sale_date'][len(df)-1]=='2018-10-29':
        shop_stop_20181029.append(i)
        s2.append(df['sale_date'][len(df)-1])

In [3]:
len(shop_stop_20181029)

250

In [4]:
ll=[]
for id in shop_stop_20181029:
    df = pd.read_csv(path+str(id)+'.csv',low_memory=False)
    ll.append(len(df))

In [5]:
ll

[225,
 558,
 558,
 446,
 366,
 331,
 513,
 177,
 362,
 497,
 357,
 372,
 188,
 389,
 359,
 380,
 434,
 118,
 85,
 360,
 410,
 80,
 184,
 214,
 132,
 366,
 501,
 131,
 223,
 295,
 218,
 215,
 371,
 221,
 224,
 414,
 305,
 413,
 188,
 340,
 79,
 252,
 145,
 186,
 145,
 144,
 163,
 118,
 155,
 210,
 184,
 290,
 101,
 216,
 184,
 106,
 173,
 132,
 157,
 172,
 183,
 214,
 148,
 236,
 147,
 147,
 291,
 219,
 44,
 117,
 264,
 171,
 182,
 164,
 237,
 114,
 148,
 229,
 122,
 145,
 124,
 168,
 158,
 128,
 167,
 157,
 150,
 125,
 163,
 158,
 96,
 80,
 83,
 501,
 123,
 84,
 150,
 152,
 110,
 146,
 155,
 106,
 188,
 147,
 133,
 98,
 124,
 110,
 478,
 98,
 111,
 61,
 152,
 118,
 123,
 70,
 129,
 112,
 49,
 152,
 70,
 127,
 147,
 103,
 147,
 64,
 103,
 124,
 97,
 131,
 70,
 125,
 120,
 136,
 117,
 98,
 151,
 102,
 147,
 144,
 71,
 147,
 96,
 146,
 48,
 54,
 94,
 52,
 147,
 43,
 98,
 63,
 107,
 86,
 54,
 49,
 45,
 80,
 59,
 95,
 76,
 49,
 106,
 84,
 39,
 95,
 113,
 67,
 60,
 121,
 58,
 14,
 200,
 46,


In [6]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

def mean_squared_error01(a,b):
    e=[]
    for i in range(len(a)):
        e1=abs((a[i]-b[i])/a[i])
        e.append(e1)
    return e

In [ ]:
# 对含有100条以上数据的店铺进行LSTM
for id in shop_stop_20181029:
    df = pd.read_csv(path+str(id)+'.csv',low_memory=False)
    if len(df)>=100:
        df=df.drop(['sale_date'],axis=1)

        values = df.values
        values = values.astype('float32')

        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled = scaler.fit_transform(values)

        # frame as supervised learning
        reframed = series_to_supervised(scaled, 1, 1)

        values = reframed.values

        train = values[:(int(0.8*len(df))), :]
        test = values[int((0.8*len(df))):, :]

        train_X, train_y = train[:, :-2], train[:, -2]
        test_X, test_y = test[:, :-2], test[:, -2]

        train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
        test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))

        model = Sequential()
        model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
        model.add(Dense(1))
        model.compile(loss='mae', optimizer='adam')

        history = model.fit(train_X, train_y, epochs=100, batch_size=8, validation_data=(test_X, test_y), verbose=2, shuffle=False)
        
        yhat = model.predict(test_X)

        yhat.reshape((len(yhat), ))

        rmse = mean_squared_error01(test_y,yhat)
        error=sum(rmse)/len(test_y)

        print(error)